## 1 - Setup

Define os caminhos padronizados para salvar os artefatos do modelo e dos mapeamentos da versão v2. 
Cria os diretórios necessários para armazenar os arquivos de modelo treinado (`modelo_char_lm_v2.keras`) e mapeamentos de caracteres (`mapeamentos_v2.pkl`).


- MODELO_OUT → versions/v2-char-lm/models/modelo_char_lm_v2.keras
- MAPEAMENTOS_OUT → versions/v2-char-lm/mappings/mapeamentos_v2.pkl

Esses caminhos organizam os arquivos de treino e facilitam reuso.

In [19]:
from pathlib import Path

BASE_DIR = Path.cwd().resolve()
if BASE_DIR.name == 'notebooks':
    BASE_DIR = BASE_DIR.parent
MODELS_DIR = BASE_DIR / 'models'
MAPPINGS_DIR = BASE_DIR / 'mappings'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
MAPPINGS_DIR.mkdir(parents=True, exist_ok=True)
MODELO_OUT = MODELS_DIR / 'modelo_char_lm_v2.keras'
MAPEAMENTOS_OUT = MAPPINGS_DIR / 'mapeamentos_v2.pkl'
print('Modelo:', MODELO_OUT)
print('Mapeamentos:', MAPEAMENTOS_OUT)


Modelo: D:\Dropbox\Coding\TCC\versions\v2-char-lm\models\modelo_char_lm_v2.keras
Mapeamentos: D:\Dropbox\Coding\TCC\versions\v2-char-lm\mappings\mapeamentos_v2.pkl


## 2 - Dados e Pré-Processamento

Esta etapa faz o download do texto completo do livro "Dom Casmurro" diretamente do Projeto Gutenberg, utilizando a função `tf.keras.utils.get_file` para garantir que o arquivo seja salvo localmente. Em seguida, o texto é carregado e decodificado para UTF-8, ignorando possíveis erros de codificação. O texto é normalizado convertendo todos os caracteres para minúsculo (`lower()`) e comprimindo múltiplos espaços em apenas um, o que reduz ruídos e inconsistências comuns em textos literários. Por fim, o tamanho total do corpus em caracteres é exibido, permitindo avaliar a quantidade de dados disponível para o treinamento do modelo de linguagem.

In [22]:
import tensorflow as tf, os, pickle

URL_LIVRO = 'https://www.gutenberg.org/files/55752/55752-0.txt'
caminho_arquivo = tf.keras.utils.get_file('dom_casmurro.txt', URL_LIVRO)
with open(caminho_arquivo, 'rb') as f:
    texto = f.read().decode('utf-8', errors='ignore')
texto = texto.lower()
texto = ' '.join(texto.split())
print('Tamanho do corpus (chars):', len(texto))

Tamanho do corpus (chars): 376658


## 3 - Vocabulário e Janelas de Treinamento

Nesta etapa, extraímos todos os caracteres únicos presentes no corpus para formar o vocabulário do modelo. Em seguida, criamos dois mapeamentos essenciais: 
- `char_to_id`: converte cada caractere em um índice inteiro único.
- `id_to_char`: faz o caminho inverso, permitindo reconstruir texto a partir de índices.

Esses mapeamentos são salvos em disco (`MAPEAMENTOS_OUT`) para garantir reprodutibilidade e facilitar o uso futuro sem necessidade de recalcular.

Definimos o comprimento da sequência de entrada (`SEQ_LEN=160`), que determina quantos caracteres o modelo observa antes de prever o próximo. Para alimentar o modelo, implementamos um gerador de batches que, sob demanda, produz pares de janelas (one-hot) e seus respectivos rótulos (próximo caractere), otimizando memória e desempenho durante o treinamento.

Por fim, calculamos o número aproximado de steps por época, considerando o tamanho do corpus e o batch size, o que auxilia no planejamento do treinamento.

In [23]:
import numpy as np

SEQ_LEN = 160
caracteres = sorted(list(set(texto)))
char_to_id = {c: i for i, c in enumerate(caracteres)}
id_to_char = {i: c for i, c in enumerate(caracteres)}
vocab = len(caracteres)
with open(MAPEAMENTOS_OUT, 'wb') as f:
    pickle.dump({'char_to_id': char_to_id, 'id_to_char': id_to_char, 'tamanho_sequencia': SEQ_LEN}, f)
print('Vocabulario:', vocab, 'caracteres')

def batch_generator(texto, char2idx, seq_len=160, batch_size=256, step=1):
    n = len(texto) - seq_len
    V = len(char2idx)
    i = 0
    while True:
        X = np.zeros((batch_size, seq_len, V), dtype=np.float32)
        y = np.zeros((batch_size,), dtype=np.int32)
        for b in range(batch_size):
            idx = (i + b*step) % n
            seq = texto[idx: idx+seq_len]
            nxt = texto[idx+seq_len]
            for t, ch in enumerate(seq):
                X[b, t, char2idx.get(ch, 0)] = 1.0
            y[b] = char2idx.get(nxt, 0)
        i = (i + batch_size*step) % n
        yield X, y

steps_per_epoch = max(1, (len(texto) - SEQ_LEN) // 256)
print('steps_per_epoch (aprox):', steps_per_epoch)

Vocabulario: 66 caracteres
steps_per_epoch (aprox): 1470


## 4 - Arquitetura do Modelo

Nesta etapa, definimos a arquitetura da rede neural responsável por aprender padrões de sequência de caracteres do texto. Utilizamos uma camada LSTM com 512 unidades, que é especialmente adequada para capturar dependências de longo prazo em dados sequenciais, como texto. A entrada do modelo é uma sequência de vetores one-hot, cada um representando um caractere do vocabulário, com dimensão (`SEQ_LEN`, `vocab`). Após processar a sequência, a LSTM gera uma representação interna que é passada para uma camada densa (`Dense`) com ativação softmax, responsável por prever a probabilidade de cada caractere possível como próximo elemento da sequência.

O modelo é compilado com o otimizador Adam, que acelera e estabiliza o treinamento, utilizando uma taxa de aprendizado inicial de 0.002 e normalização de gradiente (`clipnorm=1.0`) para evitar explosão de gradientes. A função de perda escolhida é `sparse_categorical_crossentropy`, adequada para classificação multi-classe com rótulos inteiros, e a métrica de avaliação é acurácia.

Essa configuração permite ao modelo aprender a prever o próximo caractere em uma sequência, baseando-se no contexto dos caracteres anteriores, e serve como base para geração de texto e outras tarefas de modelagem de linguagem.

In [ ]:
from tensorflow.keras import models, layers, optimizers

model = models.Sequential([
    layers.LSTM(512, input_shape=(SEQ_LEN, vocab)),
    layers.Dense(vocab, activation='softmax'),
])
optimizer = optimizers.Adam(learning_rate=2e-3, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 5 - Treinamento

Nesta etapa, o modelo é treinado para aprender padrões de sequência de caracteres do corpus. Utilizamos um gerador de batches (`batch_generator`) que alimenta o modelo com janelas de texto e seus respectivos rótulos, otimizando o uso de memória ao processar grandes volumes de dados.

O treinamento é configurado com os seguintes parâmetros:
- `BATCH_SIZE=256`: número de sequências processadas por batch.
- `EPOCHS=40`: número máximo de épocas de treinamento.

Três callbacks são utilizados para tornar o processo mais eficiente e robusto:
- `ModelCheckpoint`: salva automaticamente o melhor modelo encontrado durante o treinamento, monitorando a função de perda (`loss`).
- `ReduceLROnPlateau`: reduz a taxa de aprendizado quando a perda para de melhorar, facilitando a convergência.
- `EarlyStopping`: interrompe o treinamento caso não haja melhora na perda por várias épocas, evitando overfitting.

O método `fit` executa o treinamento usando o gerador, com o número de steps por época calculado a partir do tamanho do corpus e do batch size. Ao final, o melhor modelo é salvo no caminho especificado, pronto para ser utilizado na geração de texto.

In [ ]:
from tensorflow.keras import callbacks as kb

BATCH_SIZE = 256
EPOCHS = 40
monitor='loss'
cb = [
    kb.ModelCheckpoint(str(MODELO_OUT), save_best_only=True, monitor=monitor, mode='min'),
    kb.ReduceLROnPlateau(monitor=monitor, factor=0.5, patience=3, min_lr=5e-5),
    kb.EarlyStopping(monitor=monitor, patience=5, restore_best_weights=True),
]
gen = batch_generator(texto, char_to_id, seq_len=SEQ_LEN, batch_size=BATCH_SIZE, step=1)
steps_per_epoch = max(1, (len(texto) - SEQ_LEN) // BATCH_SIZE)
history = model.fit(gen, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, callbacks=cb, verbose=1)
print('Treinamento concluido! Melhor modelo salvo em:', MODELO_OUT)

## 6 - Salvamento e Carregamento

Nesta etapa, garantimos que os artefatos do modelo treinado e dos mapeamentos de caracteres estejam prontos para uso em geração de texto, sem necessidade de retreinar o modelo.

Primeiro, o modelo é carregado do disco usando o caminho definido em `MODELO_OUT` (caso o arquivo exista). Se o arquivo não estiver presente, utiliza-se o modelo já instanciado e treinado na sessão atual. Isso permite flexibilidade para continuar o trabalho mesmo se o notebook for reiniciado ou transferido para outro ambiente.
Em seguida, os mapeamentos de caracteres (`char_to_id` e `id_to_char`) são recarregados do arquivo `MAPEAMENTOS_OUT` usando pickle. Esses mapeamentos são essenciais para converter texto em índices inteiros (para alimentar o modelo) e para reconstruir texto a partir das previsões do modelo.

Por fim, as variáveis `modelo`, `c2i` (char_to_id) e `i2c` (id_to_char) ficam disponíveis para geração de texto, garantindo que todo o pipeline de inferência esteja pronto e reprodutível.

In [24]:
import tensorflow as tf, pickle
modelo = tf.keras.models.load_model(MODELO_OUT) if MODELO_OUT.exists() else model
with open(MAPEAMENTOS_OUT, 'rb') as f:
    maps = pickle.load(f)
c2i = maps['char_to_id']
i2c = maps['id_to_char']
print('Artefatos prontos para gerar texto.')

Artefatos prontos para gerar texto.


## 7 - Geração de Texto

Nesta etapa, implementamos a geração automática de texto usando o modelo treinado de linguagem de caracteres. O processo utiliza uma função de amostragem top-k, que limita as escolhas do próximo caractere aos k mais prováveis, tornando a geração mais criativa e menos previsível. A temperatura controla o grau de aleatoriedade: valores baixos tornam as escolhas mais conservadoras, enquanto valores altos aumentam a diversidade.

O procedimento começa com uma semente (`seed`), normalmente um trecho do corpus com cerca de 160 caracteres, que serve como contexto inicial para o modelo. A cada passo, o modelo recebe a sequência atual (codificada em one-hot) e prevê a distribuição de probabilidade para o próximo caractere. A função `sample_top_k` seleciona o próximo caractere entre os k mais prováveis, ponderando pela temperatura.

Esse ciclo se repete até atingir o comprimento desejado (`comprimento=400`), gerando texto novo que segue o estilo e padrões aprendidos do corpus original. O resultado é uma sequência textual que pode ser usada para análise, criatividade ou avaliação do modelo.

In [25]:
import numpy as np
def sample_top_k(probs, k=20, temperature=0.8, rng=None):
    probs = np.asarray(probs, dtype=np.float64)
    if rng is None: rng = np.random.default_rng()
    k = int(max(1, min(k, probs.size)))
    top_idx = np.argpartition(-probs, k-1)[:k]
    sel = probs[top_idx]
    if temperature > 0:
        logits = np.log(np.maximum(sel, 1e-9)) / temperature
        logits -= logits.max()
        sel = np.exp(logits)
    p = sel / sel.sum()
    return int(top_idx[rng.choice(len(top_idx), p=p)])

def gerar_texto(model, char_to_id, id_to_char, seed, comprimento=400, k=20, temperatura=0.8):
    vocab = len(id_to_char)
    seq_len = model.input_shape[1] if isinstance(model.input_shape, (list,tuple)) else 160
    rep = ' ' if ' ' in char_to_id else next(iter(char_to_id.keys()))
    seed = ''.join(ch if ch in char_to_id else rep for ch in seed)
    ids = [char_to_id[ch] for ch in seed][-seq_len:]
    if len(ids) < seq_len:
        pad = char_to_id.get(' ', ids[0] if ids else 0)
        ids = [pad] * (seq_len - len(ids)) + ids
    out = []
    rng = np.random.default_rng(42)
    for _ in range(comprimento):
        X = np.zeros((1, seq_len, vocab), dtype=np.float32)
        for t, idx in enumerate(ids):
            if idx < vocab: X[0, t, idx] = 1.0
        p = model.predict(X, verbose=0)[0]
        nxt = sample_top_k(p, k=k, temperature=temperatura, rng=rng)
        out.append(id_to_char.get(nxt, '?'))
        ids = ids[1:] + [nxt]
    return ''.join(out)

seed = 'os melhores meios de executar o trabalho '
print(gerar_texto(modelo, c2i, i2c, seed, comprimento=400, k=20, temperatura=0.8))

eram viver, e o ceu estava ainda no caixão e escobar e o sol claro, cuidar do cerro do dia. seguir-me alvoroçado e rir, e escobar comia uma semiração acceitei o seu perante; mas dimo-lhe o que me diseu e quando le ouvir-lhe o meu nosso suspero; de santiffeira padre-nossos e como o diabo. ezequiel apertar a minha esposa ao fim do pedaco, e vamos. este fiquei o mais. isto é, disse que, beijava-me a 
